# Spectral Clustering Specific Function Definitions

**Changelog Interval:** 3.20.23 - 3.27.23<br>
**Author:** Laura Kaplan<br>
**Purpose:** Testing randomly generated 5x5 RGB data for clustering potential<br>

# NOTES
This is a definitions file and is imported by generate_figures.ipynb

IMPORTANT! The libraries cell should be commented-out when running from the generate_figures file (or redundant imports occur)  
The libraries cell is kept here for easy-reference during debugging.



<div class="alert alert-block alert-success">

----------------------------------
# LIBRARIES
Imported in main file: generate_figures

In [1]:
# import networkx as nx
# import seaborn as sns; sns.set()
# import pylab
# %run ./def_file_general.ipynb
# %run ./def_file_spectral.ipynb

# #NUMPY
# import numpy as np
# from numpy.linalg import eig, eigh
# np.set_printoptions(precision=2,suppress=True, linewidth = 10000)


# #MATPLOTLIB
# %matplotlib inline
# import matplotlib.image as mpimg
# import matplotlib.pyplot as plt

# from mpl_toolkits.mplot3d import Axes3D

# #PIL 
# from PIL import Image, ImageOps, ImageFont, ImageDraw
# from PIL.ImageChops import add, subtract, multiply, difference, screen
# import PIL.ImageStat as stat

# #SCIPY
# import scipy as sp
# from scipy.ndimage import gaussian_filter
# import scipy.ndimage.filters as filters
# #from scipy.ndimage import affine_transform, zoom
# #from scipy import misc

# #SKIMAGE & SKLEARN
# from skimage.io import imread, imsave, imshow, show, imread_collection, imshow_collection
# from skimage import color, viewer, exposure, img_as_float, img_as_ubyte, io, data
# from skimage.transform import SimilarityTransform, warp, swirl
# from skimage.exposure import cumulative_distribution
# from skimage.util import invert, random_noise, montage
# from sklearn.cluster import KMeans, SpectralClustering
# from sklearn.decomposition import PCA

----------------------------------
# FUNCTIONS

### File & Code Management

In [2]:
def get_imports():
    for name, val in globals().items():
        if isinstance(val, types.ModuleType):
            name = val.__name__.split(".")[0]
        elif isinstance(val, type):
            name = val.__module__.split(".")[0]
            
        poorly_named_packages = {
            "PIL": "Pillow",
            "sklearn": "scikit-learn"
        }
        if name in poorly_named_packages.keys():
            name = poorly_named_packages[name]
            
        yield name

### Display & Convert

In [2]:
def display_img(image, title=''):
    if isinstance(image, Image.Image):
        _ = plt.title(title)
#         _ = plt.set_cmap('grey')
        _ = plt.axis('off')
        _ = plt.imshow(image)
    else:
        print("The image is not a PIL type")
        _ = plt.title(title)
#         _ = plt.set_cmap('grey')
        _ = plt.axis('off')
        _ = plt.imshow(Image.fromarray(image))

In [4]:
# save SINGLE image
def save_image(image, filename, color_type='rgb'):
    """
    Detects the format of the image file and saves it to disk with the given filename.

    Parameters:
        image (str, numpy.ndarray, PIL.Image, cv2.Image): The input image file, as a 
        path to an image file, a numpy array, a PIL image, or a cv2 image.
        filename (str): The desired name of the output file, including the file extension.
        color_type (str): The color mode of the input image. Valid values are 'gray', 'rgb', 'hsl', or 'hsv'. Default is 'rgb'.
    """
    # Convert the input image to a PIL image
    if isinstance(image, str):
        with Image.open(image) as im:
            pil_image = im.copy()
    elif isinstance(image, np.ndarray):
        if len(image.shape) == 2:
            # Convert grayscale image to RGB for saving as PNG or JPEG
            pil_image = Image.fromarray(image).convert(color_type.upper())
        else:
            pil_image = Image.fromarray(image, mode=color_type.upper())
    elif isinstance(image, Image.Image):
        if image.mode == 'L':
            # Convert grayscale image to RGB for saving as PNG or JPEG
            pil_image = ImageOps.grayscale(image).convert(color_type.upper())
        else:
            pil_image = image.copy()
            if image.mode != color_type.upper():
                pil_image = pil_image.convert(color_type.upper())
    elif isinstance(image, cv2.UMat):
        pil_image = Image.fromarray(cv2.UMat.get(image))
    elif isinstance(image, cv2.Mat):
        if len(image.shape) == 2:
            # Convert grayscale image to RGB for saving as PNG or JPEG
            pil_image = Image.fromarray(image).convert(color_type.upper())
        else:
            pil_image = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB), mode=color_type.upper())
    elif isinstance(image, cv2.VideoCapture):
        ret, frame = image.read()
        if frame.ndim == 2:
            # Convert grayscale image to RGB for saving as PNG or JPEG
            pil_image = Image.fromarray(frame).convert(color_type.upper())
        else:
            pil_image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB), mode=color_type.upper())
    else:
        raise ValueError(f'Invalid image type: {type(image)}')

    # Get the file format of the image
    file_format = pil_image.format

    # Save the image to disk with the given filename and format
    output_filename = filename + '.png'
    pil_image.save(output_filename)
    # with open(output_filename, 'wb') as f:
        # pil_image.save(f)

    print(f'Saved image to {output_filename}')

In [3]:
def image_to_channels(image):
    mode = image.mode
    
    if mode != 'RGB':
        print("Input image is not RGB")
        gray_array = np.array(image)
        return gray_array
    else:
        r, g, b = Image.split(image)
        r_array = np.array(r)
        g_array = np.array(g)
        b_array = np.array(b)

        return r_array, g_array, b_array

In [4]:
#create pic from arrays: single-channel OR multi-channel
def channels_to_image(channel1, channel2=None, channel3=None):
    rows, cols = channel1.shape
    
    if channel2 is None:
        return Image.fromarray(channel1)
    else:
        rgbArray = np.zeros((rows,cols,3), 'uint8')
        rgbArray[...,0] = red
        rgbArray[...,1] = green
        rgbArray[...,2] = blue
        rgbImg = Image.fromarray(rgbArray)
        return rgbImg

In [7]:
def rgb_to_3Darray(red, green, blue):
    stack = np.dstack((red, green, blue))
    return stack

In [1]:
def display_data_min_max(channel1, channel2=None, channel3=None):
    if channel2 is None and channel3 is None:
        color_setting = 'gray'
    else:
        color_setting = 'rgb'
        
        
    if color_setting == "rgb":
        print("Min red: ", np.min(channel1), "   | Min green: ", np.min(channel2), "   | Min blue: ", np.min(channel3))
        print("Max red: ", np.max(channel1), " | Max green: ", np.max(channel2), " | Max blue: ", np.max(channel3))
    elif color_setting == 'gray':
        print("Min gray: ", np.min(channel1))
        print("Max gray: ", np.max(channel1))

In [2]:
def display_data_heatmap(channel1, channel2=None, channel3=None):
    if channel2 is None and channel3 is None:
        color_setting = 'gray'
    else:
        color_setting = 'rgb'
    
    '''heatmap displays stacked numeric information. more useful for smaller datasets.'''
    if color_setting == 'rgb':
        heat_rgb = sns.heatmap(channel1+channel2+channel3, annot=True, fmt="d")
        #plt.savefig("rgb_heatmap.png", bbox_inches='tight', dpi = 100)
    elif color_setting == 'gray':
        heat_gray = sns.heatmap(channel1, annot=True, fmt="d")
        #plt.savefig("gray_heatmap.png", bbox_inches='tight', dpi = 100)
    plt.show()

In [39]:
# lo, hi, cmap = _get_display_range(image)
#display_img(img_pil, 'original image')
#display_img(img_pil, 'Array As Picture', 'rgb')
def display_data_images(channel1, channel2=None, channel3=None):
    if channel2 is None and channel3 is None:
        color_setting = 'gray'
    else:
        color_setting = 'rgb'
    
    if color_setting == 'rgb':
        '''display image at actual size'''
        pilImg = channels_to_image(channel1, channel2, channel3)
        display(pilImg)
        '''display image at increased scale'''
        stackedRGB = rgb_to_3Darray(channel1, channel2, channel3)
        plt.imshow(stackedRGB, vmin=0, vmax=255)

    elif color_setting == 'gray':
        '''display image at actual size'''
        pilImg = channels_to_image(channel1)
        display(pilImg)
        '''display image at increased scale'''
        plt.imshow(channel1, vmin=0, vmax=255, cmap='gray')

    plt.grid(False)
    plt.show()
    # lo, hi, cmap = _get_display_range(image)
    return pilImg

In [11]:
def display_initial_image_info(channel1, channel2=None, channel3=None, display_heatmap = True):
    print("Channel Shape = ", channel1.shape)
   
    if channel2 is None and channel3 is None:
        print("Pixel [2,2] Grayscale = ", channel1[2,2])
        display_data_min_max(channel1)
        pilImg = display_data_images(channel1)
        if display_heatmap == True:
            display_data_heatmap(channel1)
    else:
        print("Pixel [2,2] RGB = ", channel1[2,2], ",", channel2[2,2], ",", channel3[2,2])
        print("Pixel [22,9] RGB = ", channel1[22,9], ",", channel2[22,9], ",", channel3[22,9])
        print("Pixel [45,22] RGB = ", channel1[45,22], ",", channel2[45,22], ",", channel3[45,22])
        display_data_min_max(channel1, channel2, channel3)
        pilImg = display_data_images(channel1, channel2, channel3)
        if display_heatmap == True:
            display_data_heatmap(channel1, channel2, channel3)
    return pilImg

In [ ]:
def display_label_values(label_set, rows, cols, description=None):
    if description is None:
        description = "Labels"
    print(f"\n{description}:", label_set)
    print(f"\n{description} - Array Formatted:\n", label_set.reshape(rows, cols))

In [3]:
def display_clustering(original_img, predicted_img, labels, centers, title1, title2, mode=None, show_scatter=False):
    if mode is None:
        print("\nPlease specify whether the image should be displayed as RGB or Grayscale.")
        return False
    
    # SIDE BY SIDE PLOT
    fig, axes = plt.subplots(1, 2, figsize=(10, 5))
    if mode == 'gray':
        axes[0].imshow(original_img, cmap='gray')
    elif mode == 'rgb':
        axes[0].imshow(original_img)
    axes[0].grid(False)
    axes[0].set_title(title1)
    axes[1].imshow(predicted_img)
    axes[1].grid(False)
    axes[1].set_title(title2)
    cluster_img = transform_plot_to_image(plt)
    plt.show()
    
    if show_scatter:
        # SCATTER PLOT (contains labels and centers)
        fig = plt.figure(figsize=(5,5))
        ax = fig.add_subplot(111, projection='3d', computed_zorder=False)
        #--------FOR 2 CLUSTERS PLEASE USE:---------------
        ax.scatter(vectors[:,0], vectors[:,1], c=labels, cmap='viridis', zorder=1)
        ax.scatter(centers[:,0], centers[:,1], c='orange', marker='*', s=200, alpha=1, zorder=2)
        ax.set_xlabel('X')
        ax.set_ylabel('Y')
        ax.set_zlabel('Z')
        plt.show()
        #--------FOR 3 CLUSTERS PLEASE USE:---------------
        # ax.scatter(vectors[:,0], vectors[:,1], vectors[:,2], c=labels, cmap='viridis', zorder=1)
        # ax.scatter(centers[:,0], centers[:,1], centers[:,2], c='orange', marker='*', s=200, , alpha=1, zorder=2)
        # ax.set_xlabel('X')
        # ax.set_ylabel('Y')
        # ax.set_zlabel('Z')
        
    return cluster_img

In [9]:
# def visualize_clustering(original_img, shaped_labels, title1, title2, mode=None, display=False):
#     if mode is None:
#         raise TypeError("\nPlease specify whether the image should be displayed as RGB or Grayscale.")
    
#     # SIDE BY SIDE PLOT
#     fig, axes = plt.subplots(1, 2, figsize=(10, 5))
#     if mode == 'gray':
#         axes[0].imshow(original_img, cmap='gray')
#     elif mode == 'rgb':
#         axes[0].imshow(original_img)
#     axes[0].grid(False)
#     axes[0].set_title(title1)
#     axes[1].imshow(shaped_labels)
#     axes[1].grid(False)
#     axes[1].set_title(title2)
#     cluster_img = transform_plot_to_image(plt)
    
#     if display:
#         plt.show()
#     else:
#         plt.close(fig)
        
#     return cluster_img

In [1]:
def identify_labeled_groups(shaped_labels, k, display_num=None):
    grouped_indices = []

    # Find the indices of values that match the groupNum
    for groupNum in range(k):
        indices = [(i, j) for i, row in enumerate(shaped_labels) for j, value in enumerate(row) if value == groupNum]
        grouped_indices.append(indices)

    # Display indices
    if display_num >= k:
        raise ValueError("Error. Your display_num should be a value equal to the label of the group you'd like to observe. ")
 
    if display_num is not None:
        print(f"PIXELS IN GROUP {display_num}:")
        for i, (row, col) in enumerate(grouped_indices[display_num], start=1):
            print(f"pixel #{i}:  ({row},{col})\tRow: {row}, Column: {col}")

    return grouped_indices

In [10]:
from io import BytesIO
# def transform_plot_to_image(plot):
#     # Convert plot to image
#     buffer = BytesIO()
#     plot.savefig(buffer, format='png')
#     buffer.seek(0)
#     img = Image.open(buffer)

#     return img

def transform_plot_to_image(plot, title=None, save=False):
    # Convert plot to image
    buffer = BytesIO()
    _= plot.savefig(buffer, format='png', transparent=True, bbox_inches="tight", pad_inches=0)
    _= buffer.seek(0)
    img = Image.open(buffer)

    if save:
        if title is None:
            title = "Unnamed_Plot"
        save_image(img, f"{title}")
#         plot.savefig(title + '.png', dpi = 300, transparent=True)
        
    return img

In [13]:
def figure1_eigenvector_scatter(vectors):
    # Prepare the x-axis values: indices of the vals array
    x = np.arange(len(vectors[1]))

    # Prepare the y-axis values: values from the 2nd eigenvector
    y = vectors[1]

    # Calculate the distance from 0 for each value and normalize it
    distances = np.abs(y)
    normalized_distances = distances #/ np.max(distances)

    # Create the scatter plot
    fig = px.scatter(x=x, y=y, #color=normalized_distances, #color_continuous_scale="Viridis", 
                     labels={"x": "Index", "y": "Value", "color": "Distance from 0"},
                     title="Contents of the 2nd Eigenvector")

    # Set dark background color for the plot
    fig.update_layout({
     'plot_bgcolor': 'rgb(220, 220, 220)',  # Set the background color of the plotting area
    'paper_bgcolor': 'rgb(220, 220, 220)', # Set the background color of the entire plot

        'title': {
            'y': 0.9,
            'x': 0.5,
            'xanchor': 'center',
            'yanchor': 'top',
            'font': {'color': 'black'} # Set title font color
        },
        'xaxis': {
            'tickfont': {'color': 'black'}, # Set x-axis tick color
            'title': {'font': {'color': 'black'}} # Set x-axis title font color
        },
        'yaxis': {
            'tickfont': {'color': 'black'}, # Set y-axis tick color
            'title': {'font': {'color': 'black'}} # Set y-axis title font color
        },
        'coloraxis': {
            'colorbar': {'title': {'font': {'color': 'black'}}} # Set color bar title font color
        }
    })

    # Display the plot
    fig.show()

### Data Generation

In [38]:
def choose_predefined_data(dataset_num, seed=42, display_options = True, display_data = True):
    if display_options == True:
        print("OPTIONS GO HERE")    

    if dataset_num == 1:
        #------------------------------------------------------
        #------------------DATASET 1---------------------------
        #------------------------------------------------------
        data_desc = "5x5 Binary RGB Data (0 clusters)"  
        red = np.array([[0,0,1,1,1],[0,1,0,0,0],[0,1,1,0,0],[1,0,0,0,1],[1,0,0,1,0]])
        green = np.array([[0,1,0,1,1],[0,1,0,0,1],[0,1,1,0,1],[1,1,0,1,1],[0,1,0,1,0]])
        blue = np.array([[1,1,0,1,1],[0,0,1,0,0],[0,0,0,0,1],[0,1,1,0,0],[0,1,1,1,0]])
    elif dataset_num == 2:
        #------------------------------------------------------
        #------------------DATASET 2---------------------------
        #------------------------------------------------------
        data_desc = "2x2 RGB Data (2 clusters - half & half)" 
        red = np.array([[20, 250],[20, 250]])
        green = np.array([[20, 250],[20, 250]])
        blue = np.array([[20, 250],[20, 250]])
    elif dataset_num == 3:
        #------------------------------------------------------
        #------------------DATASET 3---------------------------
        #------------------------------------------------------
        data_desc = "3x3 RGB Data (0 clusters)"  
        red = np.array([[123, 191, 31],[218, 129, 66],[159, 239, 211]])
        green = np.array([[59,196,252],[111,15,76],[193,248,150]])
        blue = np.array([[165,90,26],[20,199,47],[39,95,230]])
    elif dataset_num == 4:
        #------------------------------------------------------
        #------------------DATASET 4---------------------------
        #------------------------------------------------------
        data_desc = "3x3 RGB Data (2 clusters - half & half)"  
        red = np.array([[100, 191, 31],[100, 129, 66],[100, 239, 211]])
        green = np.array([[100,196,252],[100,15,76],[100,248,150]])
        blue = np.array([[100,90,26],[100,199,47],[100,205,230]])
    elif dataset_num == 5:
        #------------------------------------------------------
        #------------------DATASET 5---------------------------
        #------------------------------------------------------
        data_desc = "5x5 RGB Data (1 cluster - upper left)" 
        red = np.array([[50, 50, 31, 78, 250],[50, 50, 66, 2, 34],[159, 239, 211, 74, 127],
                         [42, 86, 158, 231, 151],[108, 110, 87, 128, 20]])
        green = np.array([[50,50,252,255,9],[50,50,76,203,44],[193,248,150,205,158],
                           [122,68,195,245,126],[75,250,70,56,157]])
        blue = np.array([[50,50,26,83,129],[50,50,47,94,133],[39,95,230,30,209],
                          [62,212,65,3,225],[196,169,195,8,70]])
    elif dataset_num == 6:
        #------------------------------------------------------
        #------------------DATASET 6---------------------------
        #------------------------------------------------------
        data_desc = "5x5 RGB Data (4 clusters - 4 corners)"  
        red = np.array([[50, 50, 100, 100, 100],[50, 50, 100, 100, 100],[150,150,200,200,200],
                         [150,150,200,200,200],[150,150,200,200,200]])
        green = np.array([[50, 50, 100, 100, 100],[50, 50, 100, 100, 100],[150,150,200,200,200],
                         [150,150,200,200,200],[150,150,200,200,200]])
        blue = np.array([[50, 50, 100, 100, 100],[50, 50, 100, 100, 100],[150,150,200,200,200],
                         [150,150,200,200,200],[150,150,200,200,200]])

    elif dataset_num == 7:
        #------------------------------------------------------
        #------------------DATASET 7---------------------------
        #------------------------------------------------------
        data_desc = "6x6 RGB Data (4 clusters - 4 corners)"  
        red = np.array([[50, 50, 50, 100, 100, 100],[50, 50, 50, 100, 100, 100],[50, 50, 50, 100, 100, 100],
                        [150, 150, 150, 200, 200, 200],[150, 150, 150, 200, 200, 200],[150, 150, 150, 200, 200, 200]])
        green = np.array([[50, 50, 50, 100, 100, 100],[50, 50, 50, 100, 100, 100],[50, 50, 50, 100, 100, 100],
                        [150, 150, 150, 200, 200, 200],[150, 150, 150, 200, 200, 200],[150, 150, 150, 200, 200, 200]])
        blue = np.array([[50, 50, 50, 100, 100, 100],[50, 50, 50, 100, 100, 100],[50, 50, 50, 100, 100, 100],
                        [150, 150, 150, 200, 200, 200],[150, 150, 150, 200, 200, 200],[150, 150, 150, 200, 200, 200]])
    elif dataset_num == 8:
        #------------------------------------------------------
        #------------------DATASET 8---------------------------
        #------------------------------------------------------
        data_desc = "6x6 RGB Data + Noise (4 clusters - 4 corners)"  
        red = np.array([[ 52, 48, 48, 99, 102, 99],
        [ 48, 52, 48, 101, 101, 99],
        [ 50, 52, 49, 98, 100, 104],
        [154, 148, 150, 195, 198, 198],
        [148, 154, 152, 204, 201, 197],
        [146, 153, 152, 201, 200, 204]])

        green = np.array([[ 52, 51, 50, 103, 98, 104],
        [ 52, 51, 51, 98, 102, 102],
        [ 48, 48, 48, 100, 95, 105],
        [151, 152, 146, 195, 199, 203],
        [152, 146, 151, 199, 195, 203],
        [154, 154, 155, 198, 198, 203]])

        blue = np.array([[ 49, 46, 49, 102, 105, 100],
        [ 52, 51, 55, 100, 98, 105],
        [ 50, 53, 54, 101, 101, 105],
        [153, 152, 152, 202, 203, 195],
        [155, 152, 151, 201, 202, 197],
        [151, 151, 148, 201, 199, 202]])
    elif dataset_num == 9:
        #------------------------------------------------------
        #------------------DATASET 9---------------------------
        #------------------------------------------------------
        data_desc = "15x15 RGB Data (1 cluster - upper left)"  

        np.random.seed(seed) # set random seed for reproducibility
        # create 3 numpy arrays with random pixel values between 0 and 255
        red = np.random.randint(0, 256, size=(15, 15))
        green = np.random.randint(0, 256, size=(15, 15))
        blue = np.random.randint(0, 256, size=(15, 15))

        # set upper left 25% of each array to white
        red[:,:] = 50
        green[:,:] = 50
        blue[:,:] = 50

        # set upper left 25% of each array to white
        red[:7,:7] = 200
        green[:7,:7] = 200
        blue[:7,:7] = 200
    elif dataset_num == 10:
        #------------------------------------------------------
        #------------------DATASET 10--------------------------
        #------------------------------------------------------
        data_desc = "15x15 RGB Data (1 cluster - circle in center)"

        np.random.seed(seed) # set random seed for reproducibility
        # create 3 numpy arrays with random pixel values between 0 and 255
        red = np.random.randint(0, 256, size=(15, 15))
        green = np.random.randint(0, 256, size=(15, 15))
        blue = np.random.randint(0, 256, size=(15, 15))

        # create 3 numpy arrays with random pixel values between 0 and 50
        red = np.random.randint(0, 50, size=(15, 15))
        green = np.random.randint(0, 50, size=(15, 15))
        blue = np.random.randint(0, 50, size=(15, 15))

        # # set upper left 25% of each array to white
        # red[:,:] = 50
        # green[:,:] = 50
        # blue[:,:] = 50

        # create a meshgrid of coordinates for the array
        x, y = np.meshgrid(np.arange(red.shape[0]), np.arange(red.shape[1]))

        # compute the distance from the center of the array
        dist = np.sqrt((x - red.shape[0]/2)**2 + (y - red.shape[1]/2)**2)

        # create a binary mask for the circular region
        mask = (dist < red.shape[0]/4)

        # apply the mask to the arrays and set the values to white
        red[mask] = 200
        green[mask] = 200
        blue[mask] = 200

        # apply a Gaussian blur to make the circle less sharp
        red = gaussian_filter(red, sigma=.5)
        green = gaussian_filter(green, sigma=.5)
        blue = gaussian_filter(blue, sigma=.5)
    else:
        print(f"{dataset_num} is not a recognized predefined dataset")
        return False

#----------------------------------------------------
    rgb_combined = (red + green + blue)/3
    rows,cols = red.shape
    dist_matrix = np.zeros((rows,cols))
    
    print(f"\nSelected Dataset #{dataset_num}:", data_desc)
    
    if display_data == True:
        print("\nRED:\n", red, "\n\nGREEN:\n",  green, "\n\nBLUE:\n", blue)
    
    return red, green, blue, dist_matrix

In [ ]:
def generate_random_pixel_data(s=None, range_start=0, range_end=255):
    if s is None:
        raise ValueError("Attempted to generate random pixels. s=None not permitted.")
    else:  
        rand_values = np.random.choice((-1, 1), size=(s, s)) * np.random.randint(range_start, range_end, size=(s, s))
#         green = np.random.choice((-1, 1), size=(s, s)) * np.random.randint(range_start, range_end, size=(s, s))
#         blue = np.random.choice((-1, 1), size=(s, s)) * np.random.randint(range_start, range_end, size=(s, s))
 
    return rand_values

In [ ]:
def generate_color_test_data(size = 15, foreground_region = 'circle', background_value = 0, foreground_value = 255, 
                             add_background_noise = True, background_noise_range = 20,
                             add_foreground_noise = True, foreground_noise_range = 20, 
                             boundary_blur = True, boundary_sigma = 0.5,
                             display=False):
    
    seed = 42
    s = size
    f = foreground_value
    b = background_value
    fgn = foreground_noise_range
    bgn = background_noise_range
    bs = boundary_sigma

    print("Image data generated with the following properties: ")

    # set random seed for reproducibility
    print(f"\t- random # seed = {seed}")
    np.random.seed(seed)    
    
    #------SET IMAGE SIZE---------
    print(f"\t- resolution = {s}x{s}")
    red = np.zeros((s,s))
    green = np.zeros((s,s))
    blue = np.zeros((s,s))
    
    #------SET BACKGROUND---------
    print(f"\t- background base value = {b}")
    red[:,:] = b
    green[:,:] = b
    blue[:,:] = b
    
    if add_background_noise == True: 
        print(f"\t- background noise added. Noise range = 0-{bgn}")
        # create 3 numpy arrays with random pixel values between 0 and bgn
#         red += np.random.randint(0, bgn, size=(s, s))
#         green += np.random.randint(0, bgn, size=(s, s))
#         blue += np.random.randint(0, bgn, size=(s, s))
        red, green, blue = (channel + generate_random_pixel_data(s, 0, bgn) for channel in (red, green, blue))
        
    #------SET FOREGROUND---------
    # fill region with forced color to establish known objects
    if foreground_region == 'corner':
        print(f"\t- corner cluster created with value = {f}")
        # set upper left 25% of each array to user-specified 
        red[:s//2, :s//2] = f
        green[:s//2, :s//2] = f
        blue[:s//2, :s//2] = f

    if foreground_region == 'circle':
        print(f"\t- circle cluster created with value = {f}")
        # create a meshgrid of coordinates for the array
        x, y = np.meshgrid(np.arange(red.shape[0]), np.arange(red.shape[1]))
        # compute the distance from the center of the array
        dist = np.sqrt((x - red.shape[0]/2)**2 + (y - red.shape[1]/2)**2)
        # create a binary mask for the circular region
        mask = (dist < red.shape[0]/4)
        # apply the mask to the arrays. set value to user-specified color
        red[mask] = f
        green[mask] = f
        blue[mask] = f
    else:
        print("\t- no cluster specified. Data is entirely random.")

    if add_foreground_noise == True:
        print(f"\t- foreground noise added. Noise range = 0-{fgn}")
        # create 3 numpy arrays with random pixel values between 0 and bgn
#         red += np.random.choice((-1, 1), size=(s, s)) * np.random.randint(0, fgn, size=(s, s))
#         green += np.random.choice((-1, 1), size=(s, s)) * np.random.randint(0, fgn, size=(s, s))
#         blue += np.random.choice((-1, 1), size=(s, s)) * np.random.randint(0, fgn, size=(s, s))
        red, green, blue = (channel + generate_random_pixel_data(s, 0, fgn) for channel in (red, green, blue))
        
    if boundary_blur == True:
        print(f"\t- gaussian boundary blur applied with sigma = {bs}")
        # apply a Gaussian blur to fuzz the image regions
#         red = gaussian_filter(red, sigma = bs).astype(int)
#         green = gaussian_filter(green, sigma = bs).astype(int)
#         blue = gaussian_filter(blue, sigma = bs).astype(int)
        red, green, blue = (channel + generate_random_pixel_data(s, 0, fgn) for channel in (red, green, blue))

    #clip the values in the circumstance they exceeded the 0-255 range
    #during random number generation
    red = np.clip(red, 0, 255).astype(int)
    green = np.clip(green, 0, 255).astype(int)
    blue = np.clip(blue, 0, 255).astype(int)

    rows,cols = red.shape
    dist_matrix = np.zeros((rows,cols))
    
    if display:
        print("RED:\n", red, "\n\nGREEN:\n",  green, "\n\nBLUE:\n", blue)
    
    return red, green, blue, dist_matrix

In [2]:
# def generate_color_test_data(size = 15, forced_region = 'circle', forced_region_value = 200, 
#                        blur_boundaries = True, restrict_noise = True, noise_range = 50):
#     s = size
#     c = forced_region_value

#     # set random seed for reproducibility
#     np.random.seed(42)

#     if restrict_noise == True: 
#         # create 3 numpy arrays with random pixel values between 0 and 50
#         red = np.random.randint(0, noise_range, size=(s, s))
#         green = np.random.randint(0, noise_range, size=(s, s))
#         blue = np.random.randint(0, noise_range, size=(s, s))
        
#     else:     
#         # create 3 numpy arrays with random pixel values between 0 and 255
#         red = np.random.randint(0, 256, size=(s, s))
#         green = np.random.randint(0, 256, size=(s, s))
#         blue = np.random.randint(0, 256, size=(s, s))

#     # fill region with forced color to establish known objects
#     if forced_region == 'corner':
#         # set upper left 25% of each array to user-specified 
#         red[:,:] = c
#         green[:,:] = c
#         blue[:,:] = c
#         print("Corner Cluster Created")

#     if forced_region == 'circle':
#         # create a meshgrid of coordinates for the array
#         x, y = np.meshgrid(np.arange(red.shape[0]), np.arange(red.shape[1]))
#         # compute the distance from the center of the array
#         dist = np.sqrt((x - red.shape[0]/2)**2 + (y - red.shape[1]/2)**2)
#         # create a binary mask for the circular region
#         mask = (dist < red.shape[0]/4)
#         # apply the mask to the arrays. set value to user-specified color
#         red[mask] = c
#         green[mask] = c
#         blue[mask] = c
#         print("Center-Circle Cluster Created")
#     else:
#         print("No cluster specified. Data is entirely random.")

#     if blur_boundaries == True:
#         # apply a Gaussian blur to make the circle less sharp
#         red = gaussian_filter(red, sigma=.5)
#         green = gaussian_filter(green, sigma=.5)
#         blue = gaussian_filter(blue, sigma=.5)
#         print("Gaussian noise has been added with sigma = 0.5")

#     rows,cols = red.shape
#     dist_matrix = np.zeros((rows,cols))
#     print("RED:\n", red, "\n\nGREEN:\n",  green, "\n\nBLUE:\n", blue)
    
#     return red, green, blue, dist_matrix

In [3]:
def generate_gray_test_data(size = 15, forced_region = 'circle', forced_region_value = 200, 
                       blur_boundaries = True, restrict_noise = True, noise_range = 50):
    s = size
    rows = s
    cols = s
    v = forced_region_value

    # set random seed for reproducibility
    np.random.seed(42)
    
    # set background values with predefined noise range
    if restrict_noise == True: 
        # create 1 numpy array with random pixel values between 0 and 50
        gray = np.random.randint(0, noise_range, size=(s, s))     
    else:     
        # create 1 numpy array with random pixel values between 0 and 255
        gray = np.random.randint(0, 256, size=(s, s))

    # fill region with forced value to establish known objects
    if forced_region == 'corner':
        # set upper left 25% of each array to user-specified 
        gray[:,:] = v
        print("Corner Cluster Created")

    if forced_region == 'circle':
        # create a meshgrid of coordinates for the array
        x, y = np.meshgrid(np.arange(rows), np.arange(cols))
        # compute the distance from the center of the array
        dist = np.sqrt((x - rows/2)**2 + (y - cols/2)**2)
        # create a binary mask for the circular region
        mask = (dist < rows/4)
        # apply the mask to the arrays. set value to user-specified color
        gray[mask] = v
        print("Center-Circle Cluster Created")
    else:
        print("No cluster specified. Data is entirely random.")

    if blur_boundaries == True:
        # apply a Gaussian blur to make the circle less sharp
        gray = gaussian_filter(gray, sigma=.5)
        print("Gaussian noise has been added with sigma = 0.5")

    dist_matrix = np.zeros((rows,cols))
    print("GRAY:\n", gray)
    
    return gray, dist_matrix

In [2]:
# Helper function to draw a triangle
def draw_triangle(draw, x, y, size, fill):
    points = [(x, y), (x + size, y + size), (x - size, y + size)]
    draw.polygon(points, fill=fill)

def create_image_shapes(size=(50,50), num_shapes=3, display=True, filename='', save=False):
    # Create a random color background image
#     image = Image.new("RGB", size, "black")
    bg_color = tuple(random.randint(0, 255) for _ in range(3))
    image = Image.new("RGB", size, bg_color)
    draw = ImageDraw.Draw(image)

    # Define the shape names and their respective drawing functions
    shapes = {
        "circle": draw.ellipse,
        "square": draw.rectangle,
        "triangle": draw_triangle
    }

    # Generate three random shapes
    for _ in range(num_shapes):
        # Randomly select shape, size, placement, and color
        shape_name = random.choice(list(shapes.keys()))
        shape_size = random.randint(size[0]//6, size[0]//2)
        x = random.randint(0, size[1] - shape_size)
        y = random.randint(0, size[0] - shape_size)
        color = tuple(random.randint(0, 255) for _ in range(3))

        # Draw the shape on the image
        if shape_name == "triangle":
            shapes[shape_name](draw, x, y, shape_size, color)
        else:
            shapes[shape_name]([(x, y), (x + shape_size, y + shape_size)], fill=color)

    # Display the generated image
    if display:
        display_image(image)
    if save:
        save_image(image, f"generated_shapes_{filename}")

In [23]:
def save_data_csv(data, filename): 
    # Define the file path for the CSV file
    file_path = f"{filename}.csv"
    # Save the NumPy array to a CSV file without truncation
    np.savetxt(file_path, data, delimiter=",", fmt="%.16f")
    # Display action message
    print(f"\ndata saved successfully as {file_path}")

### FUNCTION DEFINITIONS: value adjacency

In [11]:
# R,G,B AS NEGATIVE EXPONENTIAL DISTRIBUTION --> COMBINE BY EUCLIDEAN DIST
# PRE-NORMALIZED (/255)
#---------------------- uncomment below ---------------------------------

def rgb_distance(red, green, blue):
    #retrieve list size
    n = red.shape[0] * red.shape[1]
    
    #normalize RGB values to [0,1] range
    red_norm = red.astype(float) / 255.0
    green_norm = green.astype(float) / 255.0
    blue_norm = blue.astype(float) / 255.0
    
    #create empty RGB adjacency matrices
    r_adj = np.zeros((n, n),dtype='float')
    g_adj = np.zeros((n, n),dtype='float')
    b_adj = np.zeros((n, n),dtype='float')
    rgb_adj = np.zeros((n,n),dtype='float')

    #fill adjacency matrix 
    for i in range(n):
        for j in range(i + 1, n):
            x1, y1 = i // red.shape[1], i % red.shape[1]
            x2, y2 = j // red.shape[1], j % red.shape[1]            
            r_adj[i, j] = r_adj[j, i] = np.exp(-((red_norm[x1, y1] - red_norm[x2, y2])**2))
            g_adj[i, j] = g_adj[j, i] = np.exp(-((green_norm[x1, y1] - green_norm[x2, y2])**2))
            b_adj[i, j] = b_adj[j, i] = np.exp(-((blue_norm[x1, y1] - blue_norm[x2, y2])**2))  
            
    #calculate combined adjacency list and scale values back to [0,255] range
    rgb_adj = np.sqrt(np.square(r_adj) + np.square(g_adj) + np.square(b_adj))/np.sqrt(3)
#     rgb_adj *= 255.0
    
    return rgb_adj

In [ ]:
def rgb_distance_fast(red, green, blue):
    # Retrieve list size
    n = red.shape[0] * red.shape[1]

    # Normalize RGB values to [0,1] range
    red_norm = red.astype(float) / 255.0
    green_norm = green.astype(float) / 255.0
    blue_norm = blue.astype(float) / 255.0

    # Flatten color matrices
    red_flat = red_norm.ravel()[:, np.newaxis]
    green_flat = green_norm.ravel()[:, np.newaxis]
    blue_flat = blue_norm.ravel()[:, np.newaxis]

    # Compute squared differences and exponential values for each color channel
    r_sq_diff = np.exp(-((red_flat - red_flat.T)**2))
    g_sq_diff = np.exp(-((green_flat - green_flat.T)**2))
    b_sq_diff = np.exp(-((blue_flat - blue_flat.T)**2))

    # Calculate combined adjacency matrix
    rgb_adj = np.sqrt(r_sq_diff**2 + g_sq_diff**2 + b_sq_diff**2) / np.sqrt(3)

    # Force 0's along diagonal
    np.fill_diagonal(rgb_adj, 0)

    return rgb_adj

In [12]:
# GRAY AS NEGATIVE EXPONENTIAL DISTRIBUTION 
# PRE-NORMALIZED (x/255)
#---------------------- uncomment below ---------------------------------

def gray_distance(gray):
    #retrieve list size
    n = gray.shape[0] * gray.shape[1]
    
    #normalize GRAY values to [0,1] range
    gray_norm = gray.astype(float) / 255.0

    #create empty RGB adjacency matrices
    gray_adj = np.zeros((n, n),dtype='float')

    #fill adjacency matrix 
    for i in range(n):
        for j in range(i + 1, n):
            x1, y1 = i // gray.shape[1], i % gray.shape[1]
            x2, y2 = j // gray.shape[1], j % gray.shape[1]            
            gray_adj[i, j] = gray_adj[j, i] = np.exp(-((gray_norm[x1, y1] - gray_norm[x2, y2])**2))

    return gray_adj

In [13]:
# R,G,B BY NEGATIVE EXPONENTIAL DIFF --> COMBINE BY EUCLIDEAN DIST
#---------------------- uncomment below ---------------------------------

# def rgb_distance_3_16_23(red, green, blue):
#     #retrieve list size
#     n = red.shape[0] * red.shape[1]
    
#     #create empty RGB adjacency matrices
#     r_adj = np.zeros((n, n),dtype='float')
#     g_adj = np.zeros((n, n),dtype='float')
#     b_adj = np.zeros((n, n),dtype='float')
#     rgb_adj = np.zeros((n,n),dtype='float')

#     #fill adjacency matrix 
#     for i in range(n):
#         for j in range(i + 1, n):
#             x1, y1 = i // red.shape[1], i % red.shape[1]
#             x2, y2 = j // red.shape[1], j % red.shape[1]            
#             r_adj[i, j] = r_adj[j, i] = np.exp(-((red[x1, y1] - red[x2, y2])**2))
#             g_adj[i, j] = g_adj[j, i] = np.exp(-((green[x1, y1] - green[x2, y2])**2))
#             b_adj[i, j] = b_adj[j, i] = np.exp(-((blue[x1, y1] - blue[x2, y2])**2))  
#             #print("pixel1 = (", x1, y1, ") and Pixel2 = (", x2, y2, ").")    
            
#     #calculate combined adjacency list
#     rgb_adj = np.sqrt(np.square(r_adj) + np.square(g_adj) + np.square(b_adj))/np.sqrt(3)
    
            
#     return rgb_adj

In [14]:
# R,G,B BY ABSOLUTE VALUE DIFF --> COMBINE BY NEGATIVE EXPONENTIAL DIFF
#---------------------- uncomment below ---------------------------------

# def rgb_distance(red, green, blue):
#     #retrieve list size
#     n = red.shape[0] * red.shape[1]
#     #create empty RGB adjacency matrices
#     r_adj = np.zeros((n, n),dtype='float')
#     g_adj = np.zeros((n, n),dtype='float')
#     b_adj = np.zeros((n, n),dtype='float')
#     rgb_adj = np.zeros((n,n),dtype='float')

#     #fill adjacency matrix 
#     for i in range(n):
#         for j in range(i + 1, n):
#             x1, y1 = i // red.shape[1], i % red.shape[1]
#             x2, y2 = j // red.shape[1], j % red.shape[1]            
#             r_adj[i, j] = r_adj[j, i] = abs(red[x1, y1] - red[x2, y2])
#             g_adj[i, j] = g_adj[j, i] = abs(green[x1, y1] - green[x2, y2])
#             b_adj[i, j] = b_adj[j, i] = abs(blue[x1, y1] - blue[x2, y2])
#             #print("pixel1 = (", x1, y1, ") and Pixel2 = (", x2, y2, ").")    
            
#     #calculate combined adjacency list
#     rgb_adj = 1 - np.exp(-( np.square(r_adj) + np.square(g_adj) + np.square(b_adj) ))
            
#     return rgb_adj

In [15]:
# R,G,B BY EUCLIDEAN DIST --> COMBINE BY EUCLIDEAN DIST
#---------------------- uncomment below ---------------------------------

# def rgb_distance(red, green, blue):
#     #retrieve list size
#     n = red.shape[0] * red.shape[1]
    
#     #create empty RGB adjacency matrices
#     r_adj = np.zeros((n, n),dtype='float')
#     g_adj = np.zeros((n, n),dtype='float')
#     b_adj = np.zeros((n, n),dtype='float')
#     rgb_adj = np.zeros((n,n),dtype='float')

#     #fill adjacency matrix 
#     for i in range(n):
#         for j in range(i + 1, n):
#             x1, y1 = i // red.shape[1], i % red.shape[1]
#             x2, y2 = j // red.shape[1], j % red.shape[1]
#             r_adj[i, j] = r_adj[j, i] = abs(red[x1, y1] - red[x2, y2])
#             g_adj[i, j] = g_adj[j, i] = abs(green[x1, y1] - green[x2, y2])
#             b_adj[i, j] = b_adj[j, i] = abs(blue[x1, y1] - blue[x2, y2])
    
#     #combine red, green, & blue
#     rgb_adj = np.sqrt(np.square(r_adj) + np.square(g_adj) + np.square(b_adj))/ 441.67

#     #invert values so 0 is disconnected and 1 is fully connected
#     for i in range(rows * cols):
#         for j in range(i + 1, rows * cols):
#             rgb_adj[i, j] = rgb_adj[j,i] = 1 - rgb_adj[i, j]
                     
#     return rgb_adj

In [16]:
# def rgb_distance_old(red, green, blue):
#     #convert arrays to lists
#     red_1D = np.asarray(red.flatten())
#     green_1D = np.asarray(red.flatten())
#     blue_1D = np.asarray(red.flatten())

#     #retrieve list size
#     n = red_1D.size
    
#     #create RGB adjacency matrices
#     r_adj = np.zeros((n, n),dtype='float')
#     g_adj = np.zeros((n, n),dtype='float')
#     b_adj = np.zeros((n, n),dtype='float')
#     rgb_adj = np.zeros((n,n),dtype='float')

#     # Loop over each node and compare it to every other node
#     for i in range(n):
#         for j in range(i, n):
#             #if i != j:
#             r_adj[i, j] = r_adj[j, i] = abs(red_1D[i] - red_1D[j])
#             g_adj[i, j] = g_adj[j, i] = abs(green_1D[i] - green_1D[j])
#             b_adj[i, j] = b_adj[j, i] = abs(blue_1D[i] - blue_1D[j])
    
#     #calculate combined adjacency list
#     rgb_adj = np.sqrt(np.square(r_adj) + np.square(g_adj) + np.square(b_adj))
#     #rgb_adj = np.square(r_adj) + np.square(g_adj) + np.square(b_adj)
    
#     #----------------------------normalized---------------------------------
#     print("max = ", rgb_adj.max(), "\nNon-normalized RGB_adj =\n", rgb_adj, "\n")
#     #normalize
#     rgb_adj = rgb_adj / 441.67
#     #-----------------------------------------------------------------------
    
#     return rgb_adj

In [8]:
def value_adjacency(channel1, channel2=None, channel3=None, version=0, display=False, save=False):
    if channel2 is None and channel3 is None:
        color_setting = 'gray'
    else:
        color_setting = 'rgb'
    
    if color_setting == 'rgb':
        if version == 0:
            rgb_adj = rgb_distance(red, green, blue)
        elif version == 1:
            rgb_adj = rgb_distance_fast(red, green, blue)
        else:
            raise ValueError("Version must be within selection range.",
                       "\nPlease choose from the following:",
                       "\n\t- 0 : RGB exponential calculation slow",
                       "\n\t- 1 : RGB exponential calculation fast") 
        if save:
            save_data_csv(rgb_adj, "rgb_adjacency")
        if display:
            print("-------------------------------------NORMALIZED DISTANCE RESULT-------------------------------------\n", rgb_adj)
        return rgb_adj
    
    elif color_setting == 'gray':
        gray_adj = gray_distance(gray)
        if display:
            print("-------------------------------------NORMALIZED DISTANCE RESULT-------------------------------------\n", gray_adj)
        if save:
            save_data_csv(gray_adj, "gray_adjacency")
        return gray_adj

### FUNCTION DEFINITIONS: distance adjacency

In [9]:
def euclidean_distance_0(matrix):
    rows, cols = matrix.shape
    dist_adj = np.zeros((rows * cols, rows * cols))
                     
    # Compute the Euclidean distance between each pair of pixels
    for i in range(rows * cols):
        for j in range(i + 1, rows * cols):
            x1, y1 = np.unravel_index(i, matrix.shape)
            x2, y2 = np.unravel_index(j, (rows, cols))
            dist_adj[i, j] = dist_adj[j,i] = np.sqrt((x2 - x1)**2 + (y2 - y1)**2) 
            #dist_adj[i, j] = dist_adj[j,i] = (x2 - x1)**2 + (y2 - y1)**2      
            
    #----------------------------normalized---------------------------------
    dist_adj = dist_adj / dist_adj.max()
    #-----------------------------inverted----------------------------------
    # maintains 0's along diagonal 
    for i in range(rows * cols):
        for j in range(i + 1, rows * cols):
#             dist_adj[i, j] = 1 - dist_adj[i, j]
            dist_adj[i, j] = dist_adj[j,i] = 1 - dist_adj[i, j]
            
    return dist_adj

In [10]:
#POTENTIALLY FASTER DISTANCE CALCULATION
def euclidean_distance_1(matrix):
    rows, cols = matrix.shape
    dist_adj = np.zeros((rows * cols, rows * cols))
    
    #compute the distances from every element to (x, y).
    for i in range(rows):
        for j in range(cols):
            XGrid, YGrid = np.meshgrid(np.arange(1, cols + 1), np.arange(1, rows + 1))
            distances = np.sqrt((i+1 - XGrid)**2 + (j+1 - YGrid)**2)
            #distances = (i+1 - XGrid)**2 + (j+1 - YGrid)**2
            distances = distances.flatten()
            dist_adj[(i*rows)+j,:] = distances[:]
            
    #----------------------------normalized---------------------------------
    dist_adj = dist_adj / dist_adj.max()
    #-----------------------------inverted----------------------------------
    dist_adj = 1-dist_adj
    np.fill_diagonal(dist_adj, 0)
    
    return dist_adj

In [1]:
def euclidean_distance_2(matrix):
    rows, cols = matrix.shape
    X, Y = np.indices(matrix.shape)
    X, Y = X.ravel(), Y.ravel()
    
    dist_adj = np.sqrt((X[:, np.newaxis] - X)**2 + (Y[:, np.newaxis] - Y)**2)
    #Normalize
    dist_adj = dist_adj / dist_adj.max()
    #Invert
    dist_adj = 1 - dist_adj
    #Replace 1's on diagonals with 0's 
    np.fill_diagonal(dist_adj, 0)
    
    return dist_adj

In [11]:
def distance_adjacency(dist_matrix, version=0, display=False, save=False):
    if version == 0:
        dist_adj = euclidean_distance_0(dist_matrix)
    elif version == 1:
        dist_adj = euclidean_distance_1(dist_matrix)
    elif version == 2:
        dist_adj = euclidean_distance_2(dist_matrix)
    else:
        raise ValueError("Version must be within selection range.",
                   "\nPlease choose from the following:",
                   "\n\t- 0 : euclidean distance slow",
                   "\n\t- 1 : euclidean distance fast",
                   "\n\t- 2 : euclidean distance - vectorization w. broadcasting")
        
    if display:
        print("-------------------------------------NORMALIZED DISTANCE RESULT-------------------------------------\n", dist_adj)
    if save:
        save_data_csv(dist_adj, "distance_adjacency")
        
    return dist_adj

### FUNCTION DEFINITIONS: combine matrices

In [12]:
def combine_adjacency(value_a, dist_a, alpha, display=False, save=False):
    '''Higher alpha results in more weight on color and less weight on distance.'''
    # complete_adj = (value_a + dist_a)/2
    complete_adj = ((alpha)*value_a + (1-alpha)*dist_a)
    if display:
        print("-------------------------------------COMPLETE ADJACENCY RESULT-------------------------------------\n", complete_adj)
    if save:
        save_data_csv(complete_adj, "complete_adjacency")        
    return complete_adj

### Degree & Laplacian (D & L)

In [14]:
def compute_DL(adjacency_matrix, display=False, save=False):
    
    #compute degree matrix
    D = np.diagflat(np.sum(adjacency_matrix, axis=1).T)
    
    #compute laplacian matrix
    L = D - complete_adj
    if save:
        save_data_csv(L, "laplacian_matrix")
    
    #confirm Laplacian symmetry (important for eigenvector & eigenvalue calculations)
    if np.array_equal(L, L.T):
        print("The Laplacian is confirmed to be symmetric.")
    else:
        print("WARNING! The Laplacian is NOT symmetric. Please check data.")
    
    if display:
        print("Degree Matrix:\n\n", D)
        print("Laplacian Matrix:\n\n", L)
    
    return D, L

### Eigenvalues & Eigenvectors

In [19]:
#help(eigs) #help(eigsh) #help(lobpcg)
def generate_eigens(L, eigen_type='eigh', sort=True, order='ascending', 
                    normalize=True, transpose=False, k=None, 
                    display=False, save=False):
    """
    -
    FUNCTIONS AVAILABLE: eig, eigh, eigs, eigsh, lobpcg
    MODIFICATIONS AVAILABLE: sort (ascending/descending), normalize, transpose
    NOTE! For eigs(), eigsh(), lobpcg(), please choose k.
    -
    """
    #display no-k estimator warning
    if eigen_type=='eigs' or eigen_type=='eigsh' or eigen_type=='lobpcg' and k==None:
        raise ValueError("ERROR! k cannot be None.\n",
              "To use eigenvector 'estimation' functions, please specify k number of vectors.")
        
    #track modification status for validity check
    mod_status = False
        
    print("\nGenerating Vectors with Properties...")
    
    if eigen_type == "eig":
        descriptor = "numpy_eig"
        values,vectors = eig(L)
        print(" - eig()")

    elif eigen_type == "eigh":
        descriptor = "numpy_eigh"
        values,vectors = eigh(L)  #use eigh since matrix is symmetric & real
        print(" - eigh()")
    
    elif eigen_type == "eigs":
        descriptor = "scipy_eigs"
        values, vectors = eigs(L, k)
        values = np.real(values)
        vectors = np.real(vectors)
        print(" - scipy eigs()")
        
    elif eigen_type == "eigsh":
        descriptor = 'scipy_eigsh'
        values, vectors = eigsh(L, k=k, which='LM')
        print(" - scipy eigsh() estimation")
        
    elif eigen_type == "lobpcg":
        descriptor = 'scipy_lobpcg'
        Xshape = np.random.rand(L.shape[0], k)
        #Xshape = np.array([L.shape[0], k])
        values, vectors = lobpcg(L, X=Xshape, largest=True)
        print(" - scipy lobpcg() estimation")
            
    if sort:
        # values, vectors = values[np.argsort(-values)], vectors[:, np.argsort(-values)]
        descriptor = descriptor + "_sorted"
        mod_status = True
        values,vectors = eigh(L)  #use eigh since matrix is symmetric & real
        # Sort eigenvectors by eigenvalues ASCENDING
        if order == 'ascending':
            values = values.argsort()[::1]
            vectors = vectors[:,values]
        # Sort eigenvectors by eigenvalues DESCENDING
        if order == 'descending':
            values = values.argsort()[::-1]
            vectors = vectors[:,values]
        print(f" - sorted {order}")
        
    if normalize:
        descriptor = descriptor + "_normal"
        mod_status = True
        # Using np.linalg.norm which performs L2 euclidean normalization.  
        # axis=0 performs normalization over columns.
        norms = np.linalg.norm(vectors, axis=0) 
        vectors = vectors / norms    
        print(" - normalized by linalg.norm (euclidean)")
        
#     if non_normalize:
#         descriptor = descriptor + "_normal"
#         mod_status = True
#         # Using np.linalg.norm which performs L2 euclidean normalization.  
#         # axis=0 performs normalization over columns.
#         norms = np.linalg.norm(vectors, axis=0) 
#         vectors = vectors / norms    
#         print(" - normalized by linalg.norm (euclidean)")
        
    if transpose:
        descriptor = descriptor + "_T"
        mod_status = True
        vectors = vectors.T
        print(" - transposed")
        
    # OTHER
    # descriptor = descriptor + "_misc"
    # # # vectors /= vectors[0,:]  #divide all by first element in each column
    # vectors /= vectors[:,0] #divide all by first element in each row 
    # print("\nEigenvector(s) Divided (vertical vector view):\n", vectors)
    # vectorsT = vectors.T
    # print("\nEigenvector(s) Transformed (horizontal vector view):\n", vectorsT)

    if display:
        print("\nEigenvalues:\n", values)
        print("\nEigenvectors:\n", vectors)
    
    if save: 
        # Concatenate the eigenvalues and eigenvectors into a single NumPy array
        eigen_concat = np.concatenate((values.reshape(-1, 1), vectors), axis=1)
        # Define the file path for the CSV file
        filename = f"eigenvectors_{descriptor}"
        # Save to file
        save_data_csv(eigen_concat, filename)
        
    return values, vectors, mod_status

In [21]:
"""
    Calculates the eigenvalues and eigenvectors of a matrix using the QR algorithm.
    
    Args:
    A: ndarray, the input matrix
    epsilon: float, the convergence criterion
    max_iterations: int, the maximum number of iterations
    
    Returns:
    eigenvalues: ndarray, the eigenvalues of A
    eigenvectors: ndarray, the eigenvectors of A
    """
def eigen_decomposition(A, epsilon=1e-10, max_iterations=1000):
    n = A.shape[0]
    Q = np.eye(n)
    for i in range(max_iterations):
        Q, R = np.linalg.qr(A.dot(Q))
        A_new = R.dot(Q)
        if np.abs(A - A_new).max() < epsilon:
            break
        A = A_new
    eigenvalues = np.diag(A)
    eigenvectors = Q
    return eigenvalues, eigenvectors

### Data Validity and Quality Checks

In [22]:
def check_eigenvector_validity(values, vectors, mod_status):
    """
    -
    Check the validity of the eigenvalues and eigenvectors by...
         1) diagonalizing the eigenvalues  
         2) calculating (vectors)(values)(vectors)^(-1)  
         3) comparing to original laplacian matrix. 

    IMPORTANT: This will not work on normalized or sorted eigenvalues. 
    This check should be ran on unmodified values and vectors only.
    -
    """
    if mod_status == True:
        print("COMPARISON INVALID\n",
              "Vector orders or values have been modified and are no longer comparable.")
        return False
    else: 
        # # diagonalize the eigenvalues
        values_diag = np.diag(values)
        # invert the eigenvectors
        vectors_inv = np.linalg.inv(vectors)
        #reconstruct the original matrix
        L_reconstructed = vectors @ values_diag @ vectors_inv
        #COMPARE
        laplacian_comparison(L,L_reconstructed)

In [2]:
def laplacian_comparison(matrix1, matrix2, abs_tol = 1e-08):
    '''The checking behavior is identical to the matrix_comparison method 
    but laplacian_comparison contains different print statements for the user. 
    \nThis method should be called by check_eigenvector_validity only.'''
    
    if(np.array_equal(matrix1 , matrix2)):
        print("\nL and the reconstructed L are EQUAL.",
              "\nTherefore our eigenvalues & vectors are valid.\n")
        return "PASS"
    elif(np.array_equiv(matrix1 , matrix2)):
        print("\nL and the reconstructed L are EQUIVALENT.",
              "\nThe two arrays can be broadcasted to be the same shape, with all elements are equal.\n")
        return "partial pass"
    elif(np.allclose(matrix1 , matrix2, atol=abs_tol)): #absolute tolerance between two elements for "closeness"
        print("\nL and the reconstructed L are CLOSE. Within absolute tolerance = ", abs_tol, 
              "\nThe two arrays can be broadcasted to be the same shape, and all elements are tolerably similar.\n")
        return "partial pass"
    else:
        print("\nWARNING!!!\nL and reconstructed L are NOT the same!",
              "\nConfirm that comparison is being performed on non-sorted, non-normalized eigenvectors.",
              "\nError may exist.\n")
        print("matrix 1:\n", matrix1)
        print("matrix 2:\n", matrix2)
        return "FAIL"

In [1]:
def matrix_comparison(matrix1, matrix2, abs_tol = 1e-08):

    if(np.array_equal(matrix1 , matrix2)):
        print("\nMatrix 1 and Matrix 2 are EQUAL.")
        return "PASS"
    elif(np.array_equiv(matrix1 , matrix2)):
        print("\nMatrix 1 and Matrix 2 are EQUIVALENT.",
              "\nThe two arrays can be broadcasted to be the same shape, with all elements are equal.\n")
        return "partial pass"
    elif(np.allclose(matrix1 , matrix2, atol=abs_tol)): #absolute tolerance between two elements for "closeness"
        print("\nMatrix 1 and Matrix 2 are CLOSE. Within absolute tolerance = ", abs_tol, 
              "\nThe two arrays can be broadcasted to be the same shape, and all elements are tolerably similar.\n")
        return "partial pass"
    else:
        print("\nWARNING!!!\nMatrix 1 and Matrix 2 are NOT the same!",
              "\nConfirm that comparison is being performed on non-sorted, non-normalized versions.",
              "\nError may exist. Matrices are displayed below.\n")
        print("matrix 1:\n", matrix1)
        print("matrix 2:\n", matrix2)
        return "FAIL"

### Clustering

In [40]:
# def perform_kmeans(fitting_data, k, display=True, save=True):
           
#     kmeans_model = KMeans(n_clusters=k, init='k-means++')
    
#     if fitting_data == vectors:
#         kmeans_model.fit(fitting_data[:,:k])     # FIT
#         prediction_labels = kmeans_model.predict(fitting_data[:,:k])     # PREDICT
       
#     elif fitting_data == complete_adj:
#         kmeans_model.fit_predict(complete_adj)     # FIT + PREDICT           
        
#     else:
#         print("Please specify whether you are fitting on \n'vectors' for eigenvectors, or \n'complete_adj' for adjacency")
#         return False
    
#     # RETRIEVE LABELS
#     labels = kmeans_model.labels_
    
#     # SAVE
#     if save == True:
#         save_data_csv(labels.reshape(rows, cols), f"kmeans_fit_labels_from_{fitting_data}")
#         # save_data_csv(prediction_labels.reshape(rows, cols), f"kmeans_pred_labels_from_{fitting_data}")
        
#     # DISPLAY LABELS
#     print(f"\nFitted Labels on {fitting_data}:", labels)
#     print("Fitted Labels - Array Formatted:\n", labels.reshape(rows, cols))
# #     print("\nPredicted Labels: ", prediction_labels) 
# #     print("Predicted Labels - Array Formatted:\n", prediction_labels.reshape(rows, cols))

In [1]:
def build_cluster_figures_for_vectors(k=2, min_vectors=0, max_vectors=2, label_each=False, display=False):
#     if max_vectors > k+1:
#         raise ValueError("# of vectors for training must be <= k+1")
    
    result_set = []        
#     result_set.append(pilImg)
    
    for n in range(max_vectors):
        n_vectors = n + min_vectors
        if n_vectors <= 1:# or n_vectors > k:
            continue

        # INSTANTIATE MODEL
        kmeans_model = KMeans(n_clusters=k, init='k-means++', n_init=10)          

        # FIT
        kmeans_model.fit(vectors[:, :n_vectors])       
        labels = kmeans_model.labels_

        # DISPLAY 
        results = labels.reshape(rows, cols)
        if label_each:
            result_img = retrieve_cluster_image(results, f'v={n_vectors}, k={k}')
        else:
            result_img = retrieve_cluster_image(results)
        
        result_set.append(result_img)
        
    return result_set

In [2]:
def run_clustering_test_for_vectors(k=2, min_vectors=0, max_vectors=2, display=False):
#     k = 8
#     max_vectors = k+1

    if max_vectors >= k+1:
        print("# of vectors for training must be <= k+1")
        
    result_set = []
    
    for n in range(max_vectors):
        n_vectors = n + min_vectors
        if n_vectors == 0 or n_vectors == 1:
            continue

        # INSTANTIATE MODEL
        kmeans_model = KMeans(n_clusters=k, init='k-means++', n_init=10)          

        # FIT
        kmeans_model.fit(vectors[:,1:n_vectors])       
        labels = kmeans_model.labels_

        # DISPLAY 
        results = labels.reshape(rows, cols)
        result_img = visualize_clustering(pilImg, results, "Original Image"
                                          , f"Predicted on {n_vectors} VECTORS with k={k}"
                                          , 'rgb', display=display)
        
        result_set.append(result_img)
        
    return result_set

In [3]:
def buid_cluster_figures_for_k(min_k=2, max_k=10, n_vectors=2, label_each=False, display=False):
    if n_vectors > min_k:
        raise ValueError("# of vectors for training must be <= the minimum k")
        
    result_set = []
    
    for i in range(max_k-1):
        k = i + min_k
        
        if k == 0 or k == 1:
            continue

        # INSTANTIATE MODEL
        kmeans_model = KMeans(n_clusters=k, init='k-means++', n_init=10)          

        # FIT
        kmeans_model.fit(vectors[:, :n_vectors])       
        labels = kmeans_model.labels_

        # DISPLAY 
        results = labels.reshape(rows, cols)
        if label_each:
            result_img = retrieve_cluster_image(results, f'v={n_vectors}, k={k}')
        else:
            result_img = retrieve_cluster_image(results)
        
        result_set.append(result_img)
        
    return result_set


In [4]:
def run_clustering_test_for_k(min_k=2, max_k=10, n_vectors=2, display=False):
    if n_vectors > min_k:
        raise ValueError("# of vectors for training must be <= the minimum k")
        
    result_set = []
    
    for i in range(max_k-1):
        k = i + min_k
        
        if k == 0 or k == 1:
            continue

        # INSTANTIATE MODEL
        kmeans_model = KMeans(n_clusters=k, init='k-means++', n_init=10)          

        # FIT
        kmeans_model.fit(vectors[:,1:n_vectors])       
        labels = kmeans_model.labels_

        # DISPLAY 
        results = labels.reshape(rows, cols)
        result_img = visualize_clustering(pilImg, results, "Original Image"
                                          , f"Predicted on {n_vectors} VECTORS with k={k}"
                                          , 'rgb', display=display)
        
        result_set.append(result_img)
        
    return result_set

### Pre-Existing Library Use

In [23]:
from sklearn.cluster import SpectralClustering

def segment_image(image, n_clusters):
    red, green, blue = image[:, :, 0], image[:, :, 1], image[:, :, 2]
    rows, cols = red.shape

    rgb_adj = rgb_distance(red, green, blue)
    dist_adj = euclidean_distance(red)
    complete_adj = (rgb_adj + dist_adj) / 2

    clustering = SpectralClustering(n_clusters=n_clusters, affinity='precomputed')
    labels_flat = clustering.fit_predict(complete_adj)
    labels = labels_flat.reshape(rows, cols)
    
    segmented = np.zeros_like(image)
    for i in range(n_clusters):
        segmented[labels == i] = np.median(image[labels == i], axis=0)
        
    # Plot the original image and the segmented image side by side
    fig, ax = plt.subplots(1, 2, figsize=(10, 5))
    ax[0].imshow(image)
    ax[0].set_title('Original Image')
    ax[1].imshow(segmented)
    ax[1].set_title('Segmented Image')
    plt.show()

    return segmented, labels

### Performance Metrics

In [2]:
def count_repeating_values(label_set):
#     rows, cols = label_set.shape
    all_cluster_counts = np.zeros_like(label_set, dtype=object)
    for i, ktest in enumerate(label_set): 
        for j, single in enumerate(ktest):
            # Convert the ndarray to a list and use Counter to count occurrences
            value_counts = Counter(single.flatten().tolist())
            counts = tuple(value_counts.values())
            all_cluster_counts[i,j] = counts
    return all_cluster_counts

import statistics    
def calculate_variances(pixel_counts, fileName="", save=False):
    rows, cols = pixel_counts.shape
    cluster_variances = np.zeros_like(pixel_counts, dtype=object)
    for i in range(rows): 
        for j in range(cols):
            counts = pixel_counts[i,j]
            variance = statistics.variance(counts) #for count in counts
            cluster_variances[i,j] = variance      
    if save:
        save_data_csv(cluster_variances, f'Measurements/{fileName}_variance')
#     print(cluster_variance)
    return cluster_variances 

def calculate_denominators(pixel_counts):
    rows, cols = pixel_counts.shape
    denominators = np.empty(rows)
    for i in range(rows):
        denominators[i] = sum(pixel_counts[i,0])
    return denominators


def calculate_proportions(pixel_counts, denominators):
    rows, cols = pixel_counts.shape
    cluster_proportions = np.zeros_like(pixel_counts, dtype=object)
    for i in range(rows): 
        for j in range(cols):
            counts = pixel_counts[i,j]
            proportion = tuple(count / denominators[i] for count in counts)
            cluster_proportions[i,j] = proportion
    return cluster_proportions 

def calculate_balance_measures(proportions, min_k, max_k, fileName="", save=False):
    rows, cols = proportions.shape
    cluster_counts = list(range(min_k, max_k+1))  
    balance_values = np.zeros_like(proportions, dtype=object)

    nonnorm_balance_values = np.zeros_like(proportions, dtype=object) 
    
    for i in range(rows): 
        for j in range(cols):
            for k, proportion in enumerate(proportions[i,j]):
                if k == 0:
                    unnormalized_balance = proportion
                else:   
                    unnormalized_balance *= proportion
            # norm_bal = unnorm_bal / (1/k)^k
            normalized_balance = unnormalized_balance / ((1/cluster_counts[i])**cluster_counts[i])
            balance_values[i,j] = (normalized_balance)
            nonnorm_balance_values[i,j] = (unnormalized_balance)
    if save:
        save_data_csv(balance_values, f"Measurements/{fileName}_balance")
    return balance_values, nonnorm_balance_values

def summarize_balance_measures(balance_values, measure='', fileName='', save=False):
    rows, cols = balance_values.shape
    avg_on_cols = []
    avg_lowertriangular = []
    k_sums = 0
    
    for j in range(cols):
        total_avg = sum(balance_values[:,j]) / rows
        half_avg = sum(balance_values[j:,j]) / (rows-j)
#         print(total_avg)
        avg_on_cols.append(total_avg)
        avg_lowertriangular.append(half_avg)
        
        k_sums += balance_values[j,j]
    k_avg = k_sums / rows
    avg_on_cols.append(k_avg)
    avg_lowertriangular.append(k_avg)
    
    if save:
        save_data_csv(avg_on_cols, f"Measurements/{fileName}_{measure}_avg")
        save_data_csv(avg_lowertriangular, f"Measurements/{fileName}_{measure}_avg_lower")
        
    return avg_on_cols, avg_lowertriangular

def display_balance_results(balance, summary, non_norm_bal=None, proportions=None, specific_index=None):
    if specific_index is not None:
        if proportions is not None:
            print(f"\nclustering proportions for K = {specific_index+2}\n", proportions[specific_index])
        if non_norm_bal is not None:
            print(f"non-normalized clustering balance for K = {specific_index+2}\n", non_norm_bal[specific_index])
        print(f"clustering balance for K = {specific_index+2}\n", balance[specific_index])
    
    print("\n-----------------Resulting Balances For 2nd Eigenvectors & Kth Eigenvectors-----------------")  
    rows, cols = balance.shape
    for i in range(rows):
        if balance[i,0] > balance[i,i]:
            measure_status = "2nd Eigenvector performed BETTER than k-th eigenvector."
        elif balance[i,0] < balance[i,i]:
            measure_status = "2nd Eigenvector performed WORSE than k-th eigenvector."
        else:
            measure_status = "2nd Eigenvector performed EQUIVALENTLY to the k-th eigenvector."

        print(f"FOR K = {i+2} | V=2: {balance[i,0]:<20.15f} V=k: {balance[i,i]:<20.15f}{measure_status}")
        
    print("\n-----------------Averages Across All K For Each Eigenvector Column-----------------") 
    print(summary)
    print("\tNote 1: last column corresponds to the diagonal k.\n",
          "\tNote 2: columns between the 2nd eigenvector and k appear to have smaller values than k\n",
          "\t        this is because they are inclusive of v>k results (unklike the v=k summary).")   

def calculate_clustering_performance(labels, min_k, max_k, fileName="", lower_triangular=True, save=False):
    pixel_counts = count_repeating_values(test_set_labels)
    
    variances = calculate_variances(pixel_counts, fileName=fileName, save=save)
    summary_of_var, summary_lower_var = summarize_balance_measures(variances, measure='variance', fileName=fileName, save=save)
#     display_balance_results(balance, summary_of_results, non_norm_bal, proportions)#, specific_index=0)
    
    denominators = calculate_denominators(pixel_counts)
    proportions = calculate_proportions(pixel_counts, denominators)
    balance, non_norm_bal = calculate_balance_measures(proportions, min_k=min_k, max_k=max_k, fileName=fileName, save=save)
    summary_of_results, summary_lower_triangular = summarize_balance_measures(balance, measure='balance', fileName=fileName, save=save)
    display_balance_results(balance, summary_of_results, non_norm_bal, proportions)#, specific_index=0)
    
    if lower_triangular:
        return balance, summary_lower_triangular
    else:
        return balance, summary_of_results


In [ ]:
from sklearn.neighbors import NearestNeighbors
def calculate_hopkins_statistic(data):
    n = data.shape[0]  # Number of data points

    # Step 1: Randomly select points from the dataset
    random_points = np.random.uniform(low=np.min(data, axis=0), high=np.max(data, axis=0), size=(n, data.shape[1]))

    # Step 2: Calculate nearest neighbor distances for random points
    nn_random = NearestNeighbors(n_neighbors=1).fit(random_points)
    random_distances, _ = nn_random.kneighbors(random_points)

    # Step 3: Calculate nearest neighbor distances for the original dataset
    nn_data = NearestNeighbors(n_neighbors=2).fit(data)
    data_distances, _ = nn_data.kneighbors(data)

    # Step 4: Compute the Hopkins statistic
    numerator = np.sum(random_distances)
    denominator = numerator + np.sum(data_distances[:, 1])  # Exclude self-distances
    hopkins_statistic = numerator / denominator

    return hopkins_statistic

<div class="alert alert-block alert-success">

## TEST ENVIRONMENT

In [10]:

    
def perform_eigenvector_cluster_tests(vectors, shape, min_k=2, max_k=2, min_v=2, max_v=2):
    if min_k <= 1:
            raise ValueError("Clustering cannot be performed for less than 2 classes. Please choose a higher minimum cluster value.")
    if min_v <= 1:
            raise ValueError("Clustering should not be performed using less than 2 vectors. Please choose a higher minimum vector value.")
    
    all_test_labels = np.empty((max_k-min_k+1, max_v-min_v+1), dtype=object)
    # TEST EACH CLUSTER COUNT
    current_k = min_k
    while current_k <= max_k:
        # TEST EACH EIGENVECTOR-MATRIX SIZE
        current_v = min_v
        while current_v <= max_v:
            # if current_v > current_k:
                # raise ValueError("# of vectors for training is ideally <= k")
            # INSTANTIATE MODEL
            kmeans_model = KMeans(n_clusters=current_k, init='k-means++', n_init=10)
            # FIT
            kmeans_model.fit(vectors[:, :current_v])       
            labels = kmeans_model.labels_
            results = labels.reshape(shape[0], shape[1])
            # ADD TO RESULTS
            all_test_labels[current_k-min_k, current_v-min_v] = results
            current_v += 1
        current_k += 1
            
    return all_test_labels

def retrieve_all_cluster_plots(all_labels, auto_label=False):
    all_plots = []
    rows, cols = all_labels.shape
    for i in range(rows):
        for j in range(cols):
            if auto_label:
                single_plot = retrieve_cluster_image(all_labels[i,j], f'k={i}, v={j}')
            else:
                single_plot = retrieve_cluster_image(all_labels[i,j])
            all_plots.append(single_plot)
        
    return all_plots

def retrieve_cluster_image(shaped_labels, title=None):
    plt.imshow(shaped_labels)
    if title is not None:
        plt.title(title, fontsize=12)
    plt.grid(False)
    plt.axis('off')
    plt.tight_layout(pad=0)
    cluster_img = transform_plot_to_image(plt)
    plt.close()       
        
    return cluster_img

# SUPRESS UNRELATED DEPRECIATION WARNING
import warnings
def display_clustering_tests(all_labels, all_plots, title, save_individual=False, save_set=False):
    rows, cols = all_labels.shape
    with warnings.catch_warnings():
        # warnings.simplefilter("ignore", category=FutureWarning)
        # warnings.simplefilter("ignore", category=np.VisibleDeprecationWarning)
        # Convert each PngImageFile to a NumPy array
        # all_plots_np = np.array([np.array(plot) for plot in all_plots])
        # all_plots_np = all_plots_np.reshape(rows, cols)
        # all_plots_np = np.array(all_plots).reshape(rows,cols)
        
        all_plots_np = [np.array(plot) for plot in all_plots]
        assert len(all_plots_np) == 25
        # Now pass this list of images to your plotting function
        # (Adjust plot_rowsxcols_style2 to handle a list of image arrays)
        plot_rowsxcols_style2(all_plots_np, rows, cols, title, save=save_set)

    if save_individual:
        mini_titles=[]
        result_titles=[]
        for i in range(rows):
            for j in range(cols):
                mini_titles.append(f"{title}_results{i}{j}")
            save_images(np.array(all_plots_np[i]), mini_titles)
            mini_titles=[]

    plot_rowsxcols_style2(all_plots, rows, cols, title, save=save_set)

def display_result_set(result_set, title, save_individual=False, save_set=False):
    rows = len(result_set)
    cols = len(result_set[0])
    result_titles=[]
    mini_titles=[]
    all_images=[]
    for i, result in enumerate(result_set):
        for j, img in enumerate(result_set[i]):
            all_images.append(img)
            mini_titles.append(f"{title}_results{i}{j}")
        result_titles.append(mini_titles)
        mini_titles=[]
    
    if save_individual:
        for i, result in enumerate(result_set):
            output_message = save_images(result_set[i], result_titles[i])
    plot_rowsxcols_style2(all_images, rows, cols, title, save=save_set)